<a href="https://colab.research.google.com/github/Priangshu-18/Face-Mask-Detection-System-using-CNN/blob/main/Face_Mask_Detection_System_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

In [3]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing the face mask dataset

In [4]:
# API to fetch the dataset from Kaggle
!kaggle datasets download -d omkargurav/face-mask-dataset

Dataset URL: https://www.kaggle.com/datasets/omkargurav/face-mask-dataset
License(s): unknown
 70% 115M/163M [00:00<00:00, 1.20GB/s]
100% 163M/163M [00:00<00:00, 996MB/s] 


In [5]:
# Extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/face-mask-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [6]:
!ls

data  face-mask-dataset.zip  kaggle.json  sample_data


Importing the dependencies

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mping
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split

In [9]:
with_mask_files = os.listdir('/content/data/with_mask')
print(with_mask_files[0:5])
print(with_mask_files[-5:])

['with_mask_1523.jpg', 'with_mask_2917.jpg', 'with_mask_1563.jpg', 'with_mask_1175.jpg', 'with_mask_1651.jpg']
['with_mask_234.jpg', 'with_mask_1983.jpg', 'with_mask_329.jpg', 'with_mask_2423.jpg', 'with_mask_854.jpg']


In [10]:
without_mask_files = os.listdir('/content/data/without_mask')
print(without_mask_files[0:5])
print(without_mask_files[-5:])

['without_mask_3569.jpg', 'without_mask_41.jpg', 'without_mask_29.jpg', 'without_mask_2666.jpg', 'without_mask_3328.jpg']
['without_mask_3451.jpg', 'without_mask_73.jpg', 'without_mask_2238.jpg', 'without_mask_388.jpg', 'without_mask_2261.jpg']
